# Validation of point_layer point_variable 

In [ ]:
# bin_value using function from r4ecology's github
import numpy as np
def bin_value(x, bin_res):
    return np.floor((x + bin_res / 2) / bin_res + 0.5) * bin_res - bin_res / 2


In [ ]:
chunk_start

In [ ]:
variable = "point_variable".lower()
layer = "point_layer"
# get the units. File inspection could be randomized in case people have put loose files in there...
import glob
df = pd.read_csv("../../matched/mapping.csv")
df = df.query("variable == @variable")
pattern = list(df.pattern)[0]
# while True:
#     i = 0
#     patterns = pattern.split("/")
#     for x in patterns:
#         if x == "**":
#             break
#         i+=1
#     new_pattern = glob.glob("/".join(patterns[0:i])+"/" + "**" )[-1].split("/")[-1]
#     patterns[i] = new_pattern
#     pattern = "/".join(patterns)

    
#     if len([x for x in pattern.split("/") if x == "**"]) == 0:
#         break
paths = pd.read_csv(glob.glob(f"../../matched/point/**/{layer}/{variable}/paths.csv")[0]).path

for ff in paths:
    try:
        ds = nc.open_data(paths[0])
        model_variable = list(df.model_variable)[0].split("+")[0]
        unit = list(ds.contents.query("variable == @model_variable").unit)[0]
        break
    except:
        pass
            

## Read in the data

In [ ]:
ff = glob.glob(f"../../matched/point/**/{layer}/{variable}/*_{variable}.csv")[0]
vv_source = os.path.basename(ff).split("_")[0]
df = pd.read_csv(ff)
if variable == "ph":
    df = df.query("observation > 4").reset_index(drop = True)
# Danish part is always dubious
df = df.query("lon < 9")
ds= nc.open_data(f"{data_dir}/amm7_val_subdomains.nc")
ds.subset(variable = "Shelf")
ds.as_missing(0)
ds.regrid(df.loc[:,["lon", "lat"]].drop_duplicates().reset_index(drop = True), "nn")
df_grid = ds.to_dataframe().reset_index().dropna().drop_duplicates()
df = df.merge(df_grid)
df_locs = df.loc[:,["lon", "lat"]].drop_duplicates()
# bin to 0.01 resolution
df_raw = df
df["lon"] = df["lon"].apply(lambda x: bin_value(x, 0.5))
df["lat"] = df["lat"].apply(lambda x: bin_value(x, 0.5))
if variable != "carbon":
    df = df.groupby(["lon", "lat", "year", "month"]).mean().reset_index()
else:
    df = df.groupby(["lon", "lat"]).mean().reset_index()

In [ ]:
from IPython.display import Markdown as md

if variable not in ["poc", "doc", "carbon"]:

    if layer == "bottom":
        md(f"Near-bottom values of {variable} were extracted from ICES bottle and CTD data.")
    else:
        md(f"Values from the top 5 m of the water column were extracted from ICES bottle and CTD data.")

if layer == "bottom":
    md(f"This data was extracted from vertical profiles. The near_bottom value was defined as the value closest to the bottom, that was within 5 m of the bottom. Bathymetry was estimated using GEBCO Bathymetry data.")
else:
    md(f"This data was extracted from vertical profiles. Values from the top 5 m were extracted from the database. This was compared with the model values from the surface level.")

md(f"In total there were {len(df)} {layer} values extracted from the observational database.")

if layer == "bottom":
    md("**Note:** this analysis has been restricted to observations on the shelf region.")

if variable in ["carbon"]:
    md("Carbon data was compiled from multiple sources")

if variable == "poc":
    md("Particulate organic carbon data was compiled from multiple sources")

In [ ]:
# bottom 1% of observations
bot_low = df.observation.quantile(0.001)
df = df.query(f"observation >= {bot_low}")

In [ ]:
%%capture --no-display
%%R -i df_locs -i variable -i unit -w 1000 -h 1200
library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
# get lon, lat limits from profile_mld

xlim = c(min(df_locs$lon), max(df_locs$lon))
ylim = c(min(df_locs$lat), max(df_locs$lat))



gg <- df_locs %>%
# final six months of the year
    ggplot()+
    geom_point(aes(lon, lat))+
    theme_gray(base_size = 24)+
    # add colour scale. Minimum zero, label 100, ">100"
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) 

# figure out if lon minimum is less than -10
if( min(df_locs$lon) < -10 ){
    # add sensible labels for longitude and latitude

    gg <- gg +
    scale_x_continuous(breaks = seq(-10, 5, 5), labels = c("10°W", "5°W", "0°", "5°E"))+ 
    scale_y_continuous(breaks = seq(45, 60, 5), labels = c("45°N", "50°N", "55°N", "60°N"))+
    labs(x = "", y = "") 


}

    # move legen

gg

In [ ]:
md(f"**Figure {i_figure}:** Map of {layer} {variable} observations from ICES.")

In [ ]:
%%R -i df -i variable -i unit -w 1000 -h 1200
# calculate number of observations per month
# figure out if "month" in df
if("month" %in% colnames(df)){

df1 <- df %>%
    group_by(lon, lat, month) %>%
    summarise(observation = n()) %>%
    ungroup()   

# plot number of observations per month using plotnine and geom_bar

# change month to month name
df1$month <- factor(df1$month, levels = 1:12, labels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))

gg <- df1 %>%
    ggplot(aes(x = month, y = observation))+
    theme_gray(base_size = 24)+
    geom_bar(stat = "identity")+
    labs(y = "Number of observations", x= "")

gg
}

In [ ]:
md(f"**Figure {i_figure}:** Number of {layer} observations per month for {variable}.")
i_figure += 1

In [ ]:
%%capture --no-display
%%R -i df -i variable -i unit -i layer -w 1000 -h 1200
#%%R -i df -i variable -i unit -w 1600 -h 1000
if("month" %in% colnames(df)){


library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
# get lon, lat limits from profile_mld

xlim = c(min(df$lon), max(df$lon))
ylim = c(min(df$lat), max(df$lat))


df <- df %>%
    mutate(bias = model - observation) 

# calculate the absolate bias

df1 <- df %>%
    mutate(bias = abs(bias))
# calculate the 98th percentile of the absolute bias
bias_high <- df1$bias %>% quantile(0.98)
# cap the bias to +/1 98th percentile
df$bias[df$bias > bias_high] <- bias_high
df$bias[df$bias < -bias_high] <- -bias_high


# # convert month number to month in profile_mld
df <- df %>%
    arrange(month)
df$month <- factor(df$month, levels = df$month, labels = month.abb[df$month])
# df$month <- factor(df$month, labels = month.abb)

title <- str_glue("Bias in {layer} {variable} ({unit})")

out = str_glue("../../results/{layer}/{variable}/{layer}_{variable}_bias.csv")

# # check directory exists for out
if (!dir.exists(dirname(out))){
    dir.create(dirname(out), recursive = TRUE)
}
df %>% write_csv(out)

# df.to_csv(out, index = False)

# export to csv



gg <- df %>%
# final six months of the year
    ggplot()+
    geom_raster(aes(lon, lat, fill = bias))+
    facet_wrap(~month)+
    theme_gray(base_size = 24)+
    # add colour scale. Minimum zero, label 100, ">100"
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    # move legend to the top. Make it 3 cm wide
    # move legend title to the bottom and centre it
    scale_fill_gradient2(low = "blue", high = "red",
                       guide = guide_colorbar(title.position = "bottom", title.hjust = 0.5, title.theme = element_text(angle = 0, size = 20, family = "Helvetica"))
  )+
    theme(
    legend.position = "bottom", legend.direction = "horizontal", legend.box = "horizontal", legend.key.width = unit(6.0, "cm"),
    legend.key.height = unit(1.0, "cm"))+
    # set the legend title to bias
    labs(fill = title)

# figure out if lon minimum is less than -10
if( min(df$lon) < -10 ){
    # add sensible labels for longitude and latitude

    gg <- gg +
    scale_x_continuous(breaks = seq(-10, 5, 5), labels = c("10°W", "5°W", "0°", "5°E"))+ 
    scale_y_continuous(breaks = seq(45, 60, 5), labels = c("45°N", "50°N", "55°N", "60°N"))+
    labs(x = "", y = "") 


}

    # move legen

gg
}

In [ ]:
if variable not in ["carbon"]:
    md(f"**Figure {i_figure}**: Bias in {layer} {variable}. The bias is calculated as model - observation. The colour scale is from blue (negative bias) to red (positive bias). The colour scale is capped at the 98th percentile of the absolute bias. This is to avoid a few extreme outliers from dominating the colour scale. **Note:** values have been binned and averaged to 0.5 degree resolution.") 
    i_figure += 1

In [ ]:
%%capture --no-display
%%R -i df_raw -i variable -i unit -w 1000 -h 1200
#%%R -i df -i variable -i unit -w 1600 -h 1000
if("month" %in% colnames(df)){


library(tidyverse, warn.conflicts = FALSE)

# # convert month number to month in profile_mld
df <- df_raw %>%
    arrange(month)
# df$month <- factor(df$month, labels = month.abb)

x_lab <- str_glue("Model {variable} ({unit})")
y_lab <- str_glue("Observed {variable} ({unit})")


gg <- df %>%
# final six months of the year
    ggplot()+
    geom_point(aes(model, observation))+
    facet_wrap(~month)+
    theme_gray(base_size = 24)+
    labs(fill = title)+
    geom_abline()+
    geom_smooth(aes(model, observation), method = "gam")+
    labs(x = x_lab, y = y_lab)
    # move legen

gg
}

In [ ]:
if variable not in ["carbon"]:
    md(f"**Figure {i_figure}**: Model vs observed {variable} for {layer} observations. The model is the ICES model and the observations are from ICES. The line is a GAM fit to the data. The data has been binned to 0.5 degree resolution.")
    i_figure = i_figure + 1

In [ ]:
%%capture --no-display
%%R -i df_raw -i variable -i unit -w 1000 -h 1200
#%%R -i df -i variable -i unit -w 1600 -h 1000

library(tidyverse, warn.conflicts = FALSE)




x_lab <- str_glue("Model {variable} ({unit})")
y_lab <- str_glue("Observed {variable} ({unit})")


gg <- df %>%
# final six months of the year
    ggplot()+
    geom_point(aes(model, observation))+
    theme_gray(base_size = 24)+
    labs(fill = title)+
    geom_abline()+
    geom_smooth(aes(model, observation), method = "gam")+
    labs(x = x_lab, y = y_lab)
    # move legen

gg

In [ ]:
md(f"**Figure {i_figure}**: Model vs observed {variable} for {layer} values. The observations are from ICES bottle and CTD data. 5 degree resolution. The line is a GAM fit to the data. The shaded area is the 95% confidence interval of the GAM fit.")
i_figure += 1

## Summary statistics

In [ ]:
md(f"The overall ability of the model to predict the observed {variable} was assessed by calculating the average bias, the root mean square error (RMSE) and the correlation coefficient (R). The bias was calculated as the model value minus the observed value. The RMSE was calculated as the square root of the mean squared error. The correlation coefficient was calculated as the Pearson correlation coefficient between the model and observed values.") 
md(f"This was calculated for each month and for the entire dataset. The results are shown in the tables below.")
md(f"This is calculated in two separate ways. First, we use the raw model and observed values. Second, we use data that was averaged to 0.5 to 0.5 bins to account for spatial bias.")

In [ ]:
if variable not in ["carbon"]:
    df_bias = (
        df_raw
        .assign(bias = lambda x: x.model - x.observation)
        .groupby("month")
        .mean()
        .reset_index()
        .loc[:,["month", "bias"]]
        # convert month number to name
        .assign(month = lambda x: x.month.apply(lambda y: calendar.month_abbr[y]))
    )
    # add average bias to df_bias as a separate row
    annual_bias = df_raw.model.mean() - df_raw.observation.mean() 
    df_bias = pd.concat([df_bias, pd.DataFrame({"month": ["All"], "bias": [annual_bias]})])

    # move the final row to the top
    df_bias = pd.concat([df_bias.iloc[[-1]], df_bias.iloc[:-1]])
else:
    # only want annual
    df_bias = pd.DataFrame({"month": ["All"], "bias": [df_raw.model.mean() - df_raw.observation.mean()]})
if variable not in ["carbon"]:
    # now create an rmse dataframe
    df_rmse = (
        df_raw
        .assign(month = lambda x: x.month.apply(lambda y: calendar.month_abbr[y]))
        .groupby("month")
        .apply(lambda x: np.sqrt((x.model - x.observation).pow(2).mean()))
        .reset_index()
        .rename(columns={0: "rmse"})
    )
    # add average rmse to df_rmse as a separate row
    annual_rmse = np.sqrt(((df_raw.model - df_raw.observation).pow(2)).mean())
    df_rmse = pd.concat([df_rmse, pd.DataFrame({"month": ["All"], "rmse": [annual_rmse]})])
    # move the final row to the top
    df_rmse = pd.concat([df_rmse.iloc[[-1]], df_rmse.iloc[:-1]])
else:
    # only want annual
    df_rmse = pd.DataFrame({"month": ["All"], "rmse": [np.sqrt(((df_raw.model - df_raw.observation).pow(2)).mean())]})
# rename the month column to Month
# merge the two dataframes
df_table = copy.deepcopy(df_bias).merge(df_rmse)
df_table = df_table.round(2)
# create df_corr
if variable not in ["carbon"]:
    df_corr = (
        df_raw
        .groupby("month")
        .apply(lambda x: x.model.corr(x.observation))
        .reset_index()
        .rename(columns={0: "correlation"})
        .assign(month = lambda x: x.month.apply(lambda y: calendar.month_abbr[y]))
    )
    # add average correlation to df_corr as a separate row
    # calculate annual correlation using all data
    annual_corr = df_raw.model.corr(df_raw.observation)
    df_corr = pd.concat([df_corr, pd.DataFrame({"month": ["All"], "correlation": [annual_corr]})])
    # df_corr = df_corr.append({"month": "All", "correlation": annual_corr}, ignore_index=True)

    # move the final row to the top
    df_corr = pd.concat([df_corr.iloc[[-1]], df_corr.iloc[:-1]])
else:
    # only want annual
    df_corr = pd.DataFrame({"month": ["All"], "correlation": [df_raw.model.corr(df_raw.observation)]})
df_table = df_table.merge(df_corr)
df_table = df_table.round(2)
df_table = df_table.rename(columns={"month": "Month", "bias": "Bias", "rmse": "RMSE", "correlation": "Correlation"})
df_table = df_table[["Month", "Bias", "RMSE", "Correlation"]]
# change Month to Period
df_table = df_table.rename(columns={"Month": "Time period"})

if variable not in ["carbon"]:
    # add commas to bias and rmse
    df_number = df_raw.groupby("month").count().reset_index().loc[:,["month", "observation"]]
# convert month number to name
    df_number["month"] = df_number["month"].apply(lambda x: calendar.month_abbr[x])
    df_number = df_number.rename(columns={"month": "Time period", "observation": "Number of observations"})
else:
    df_number = pd.DataFrame({"Time period": ["All"], "Number of observations": [len(df_raw)]})

# add total number of observations
annual_number = len(df_raw)
if variable not in ["carbon"]:
    df_number = pd.concat([df_number, pd.DataFrame({"Time period": ["All"], "Number of observations": [annual_number]})])
# df_number = df_number.append({"Time period": "All", "Number of observations": annual_number}, ignore_index=True)
df_table = df_table.merge(df_number)

# include commas in the number of observations
df_table["Number of observations"] = df_table["Number of observations"].apply(lambda x: "{:,}".format(x))

df_table.style.hide(axis="index")

In [ ]:
md(f"**Table {i_table}:** Average bias and root-mean square error in {layer} {variable} for each month using the raw ICES data. The bias is calculated as model - observation. The average bias is calculated as the mean of the monthly biases.")
i_table += 1

In [ ]:
if variable not in ["carbon"]:
    df_bias = (
        df
        .assign(bias = lambda x: x.model - x.observation)
        .groupby("month")
        .mean()
        .reset_index()
        .loc[:,["month", "bias"]]
        # convert month number to name
        .assign(month = lambda x: x.month.apply(lambda y: calendar.month_abbr[y]))
    )
    # add average bias to df_bias as a separate row
    annual_bias = df.model.mean() - df.observation.mean() 
    df_bias = pd.concat([df_bias, pd.DataFrame({"month": ["All"], "bias": [annual_bias]})])

    # move the final row to the top
    df_bias = pd.concat([df_bias.iloc[[-1]], df_bias.iloc[:-1]])
else:
    # only want annual
    df_bias = pd.DataFrame({"month": ["All"], "bias": [df.model.mean() - df.observation.mean()]})
if variable not in ["carbon"]:
    # now create an rmse dataframe
    df_rmse = (
        df
        .assign(month = lambda x: x.month.apply(lambda y: calendar.month_abbr[y]))
        .groupby("month")
        .apply(lambda x: np.sqrt((x.model - x.observation).pow(2).mean()))
        .reset_index()
        .rename(columns={0: "rmse"})
    )
    # add average rmse to df_rmse as a separate row
    annual_rmse = np.sqrt(((df.model - df.observation).pow(2)).mean())
    df_rmse = pd.concat([df_rmse, pd.DataFrame({"month": ["All"], "rmse": [annual_rmse]})])
    # move the final row to the top
    df_rmse = pd.concat([df_rmse.iloc[[-1]], df_rmse.iloc[:-1]])
else:
    # only want annual
    df_rmse = pd.DataFrame({"month": ["All"], "rmse": [np.sqrt(((df.model - df.observation).pow(2)).mean())]})
# rename the month column to Month
# merge the two dataframes
df_table = copy.deepcopy(df_bias).merge(df_rmse)
df_table = df_table.round(2)
# create df_corr
if variable not in ["carbon"]:
    df_corr = (
        df
        .groupby("month")
        .apply(lambda x: x.model.corr(x.observation))
        .reset_index()
        .rename(columns={0: "correlation"})
        .assign(month = lambda x: x.month.apply(lambda y: calendar.month_abbr[y]))
    )
    # add average correlation to df_corr as a separate row
    # calculate annual correlation using all data
    annual_corr = df.model.corr(df.observation)
    df_corr = pd.concat([df_corr, pd.DataFrame({"month": ["All"], "correlation": [annual_corr]})])
    # df_corr = df_corr.append({"month": "All", "correlation": annual_corr}, ignore_index=True)

    # move the final row to the top
    df_corr = pd.concat([df_corr.iloc[[-1]], df_corr.iloc[:-1]])
else:
    # only want annual
    df_corr = pd.DataFrame({"month": ["All"], "correlation": [df.model.corr(df.observation)]})
df_table = df_table.merge(df_corr)
df_table = df_table.round(2)
df_table = df_table.rename(columns={"month": "Month", "bias": "Bias", "rmse": "RMSE", "correlation": "Correlation"})
df_table = df_table[["Month", "Bias", "RMSE", "Correlation"]]
# change Month to Period
df_table = df_table.rename(columns={"Month": "Time period"})

if variable not in ["carbon"]:
    # add commas to bias and rmse
    df_number = df.groupby("month").count().reset_index().loc[:,["month", "observation"]]
# convert month number to name
    df_number["month"] = df_number["month"].apply(lambda x: calendar.month_abbr[x])
    df_number = df_number.rename(columns={"month": "Time period", "observation": "Number of observations"})
else:
    df_number = pd.DataFrame({"Time period": ["All"], "Number of observations": [len(df)]})

# add total number of observations
annual_number = len(df)
if variable not in ["carbon"]:
    df_number = pd.concat([df_number, pd.DataFrame({"Time period": ["All"], "Number of observations": [annual_number]})])
# df_number = df_number.append({"Time period": "All", "Number of observations": annual_number}, ignore_index=True)
df_table = df_table.merge(df_number)

# include commas in the number of observations
df_table["Number of observations"] = df_table["Number of observations"].apply(lambda x: "{:,}".format(x))

df_table.style.hide(axis="index")

In [ ]:
md(f"**Table {i_table}:** Average bias and root-mean square error in {layer} {variable} for each month using binned ICES data. ICES data was averaged in each 0.5 by 0.5 degree cell in each year and month. The bias is calculated as model - observation. The average bias is calculated as the mean of the monthly biases.")
i_table += 1

## Linear regresion analysis of model vs observed point_variable

In [ ]:
md(f"A linear regression analysis of modelled and observed {variable} was performed. The modelled {variable} was used as the independent variable and the observed {variable} was used as the dependent variable. The results are shown in the table below.")

md("The regression was carried out using the Python package statsmodels.")

In [ ]:

# do a linear regression of model vs observed in df
X = df.model.values
Y = df.observation.values
# linear regression using statsmodels
import statsmodels.api as sm
X = sm.add_constant(X)
# make X and Y random numbers between 0 and 1
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# get the slope and intercept
intercept, slope = model.params
# calculate the r squared
r2 = model.rsquared
# calculate the p value of the slope
p = model.f_pvalue

p = model.f_pvalue
# put that in a dataframe
df_stats = pd.DataFrame({"Slope": slope, "Intercept": intercept, "R2": r2, "P": p}, index = ["All"]).assign(Period = "All")
# do this month by month append to df_stats

for month in range(1, 13):
    try:
        X = df.query("month == @month").model.values
        Y = df.query("month == @month").observation.values
        X = sm.add_constant(X)
        model = sm.OLS(Y, X).fit()
        intercept, slope = model.params
        r2 = model.rsquared
        p = model.f_pvalue
        df_stats = pd.concat([df_stats, pd.DataFrame({"Slope": slope, "Intercept": intercept, "R2": r2, "P": p}, index = [month]).assign(Period = month)])
        df_stats.loc[df_stats.index[-1], "Period"] = calendar.month_abbr[month]
    except:
        pass
# sort period appropriately, so All is first then ordered by month
df_stats["Period"] = pd.Categorical(df_stats["Period"], [calendar.month_abbr[x] for x in range(1, 13)] + ["All"])
# round p-value to 3 dp
df_stats["P"] = df_stats["P"].round(5)
# change P to p-value
df_stats = df_stats.rename(columns={"P": "p-value"})
# put Period first
df_stats = df_stats[["Period", "Slope", "Intercept", "R2", "p-value"]]
# 

df_stats.style.hide(axis="index")

In [ ]:
chunk_end

## Data citations

In [ ]:
if variable == "poc":
    md("Boss, Emmanuel; Picheral, Marc; Searson, Sarah; Le Goff, Hervé; Reverdin, Gilles; Leeuw, Thomas; Chase, Alison P; Bricaud, Annick; Kolber, Zbigniew S; Taillandier, V; Pesant, Stephane; Tara Oceans Consortium, Coordinators; Tara Oceans Expedition, Participants (2017): Underway surface water data during the Tara Oceans expedition in 2009-2012 [dataset]. PANGAEA, https://doi.org/10.1594/PANGAEA.873566, In: Boss, Emmanuel; Picheral, Marc; Searson, Sarah; Marec, Claudie; Le Goff, Hervé; Reverdin, Gilles; Leeuw, Thomas; Chase, Alison P; Anderson, Leif G; Gattuso, Jean-Pierre; Pino, Diana Ruiz; Padín, Xose Antonio; Grondin, Pierre-Luc; Matuoka, Atsushi; Babin, Marcel; Bricaud, Annick; Kolber, Zbigniew S; Taillandier, V; Hafez, Mark; Chekalyuk, Alexander; Pesant, Stephane; Météo France; Tara Oceans Consortium, Coordinators (2017): Harmonised data from underway navigation, meteorology and surface water measurements during the Tara Oceans expedition in 2009-2013 [dataset publication series]. PANGAEA, https://doi.org/10.1594/PANGAEA.873592")
    md("Röttgers, Rüdiger; Bi, Shun; Burmester, Henning; Heymann, Kerstin; Hieronymi, Martin; Krasemann, Hajo; Schönfeld, Wolfgang (2023): Water inherent optical properties and concentrations of water constituents from the German Bight and adjacent regions: concentrations and auxiliary data [dataset]. PANGAEA, https://doi.org/10.1594/PANGAEA.950767, In: Röttgers, Rüdiger; Bi, Shun; Burmester, Henning; Heymann, Kerstin; Hieronymi, Martin; Krasemann, Hajo; Schönfeld, Wolfgang (2023): Water inherent optical properties and concentrations of water constituents from the German Bight and adjacent regions [dataset bundled publication]. PANGAEA, https://doi.org/10.1594/PANGAEA.950774)")
    md("Loisel, Hubert; Duforêt-Gaurier, Lucile; Tran, Trung Kien; Jorge, Daniel S F; Steinmetz, Francois; Mangin, Antoine; Bretagnon, Marine; d'Andon, Odile (2023): Database (DSM) of in situPOC, SPM and Rrs collected between 1997 and 2018 [dataset]. PANGAEA, https://doi.org/10.1594/PANGAEA.960962")
    md("Lønborg, Christian; Carreira, Cátia; Abril, Gwenael; Agustí, Susana; Amaral, Valentina; Andersson, Agneta; Arístegui, Javier; Bhadury, Punyasloke; Bif, Mariana B; Borges, Alberto Vieira; Bouillon, Steven; Calleja, Maria Ll; Cotovicz, Luiz C Jr; Cozzi, Stefano; Doval, Maryló; Duarte, Carlos Manuel; Eyre, Bradley D; Fichot, Cedric; García-Martín, Elena; Garzon-Garcia, Alexandra; Giani, Michele; Gonçalves-Araujo, Rafael; Gruber, Renee K; Hansell, Dennis A; Hashihama, Fuminori; He, Ding; Holding, Johnna M; Hunter, William Ross; Ibánhez, J Severino; Ibello, Valeria; Jiang, Shan; Kim, Guebuem; Klun, Katja; Kowalczuk, Piotr; Kubo, Atsushi; Lee, Choon Weng; Lopes, Claudia B; Maggioni, Federica; Magni, Paolo; Marrasé, Celia; Martin, Patrick; McCallister, S Leigh; McCallum, Rosh; M Medeiros, Patricia; G Morán, Xosé Anxelu; Muller-Karger, Frank; Myers-Pigg, Allison; Norli, Marit; Oakes, Joanne M; Osterholz, Helena; Park, Hyekyung; Lund Paulsen, Maria; Rosentreter, Judith A; Ross, Jeff; Rueda-Roa, Digna; Santinelli, Chiara; Shen, Yuan; Teira, Eva; Tinta, Tinkara; Uher, Guenther; Wakita, Masahide; Ward, Nicholas D; Watanabe, Kenta; Xin, Yu; Yamashita, Youhei; Yang, Liyang; Yeo, Jacob; Yuan, Huamao; Zheng, Qiang; Álvarez‐Salgado, Xosé Antón (2023): A global database of dissolved organic matter (DOM) concentration measurements in coastal waters (CoastDOM v.1) [dataset]. PANGAEA, https://doi.org/10.1594/PANGAEA.964012")


In [ ]:
if vv_source == "ices":
    md(f"Data for {variable} was downloaded from the ICES website on 3rd March 2023. The data is available from [ICES](https://data.ices.dk/view-map).") 
    md("A minimum requirement citation would include the following: “ICES Data Portal, Dataset on Ocean Hydrochemistry, 2023. ICES, Copenhagen”")

    md("Extended citation: ICES Data Portal, Dataset on Ocean HydroChemistry, Extracted March 3, 2023. ICES, Copenhagen")

In [ ]:
if variable == "carbon":
    md('Diesing, Markus, Terje Thorsnes, and Lilja Rún Bjarnadóttir. "Organic carbon densities and accumulation rates in surface sediments of the North Sea and Skagerrak." Biogeosciences 18.6 (2021): 2139-2160.')